In [1]:
!wget http://www.eraserbenchmark.com/zipped/movies.tar.gz
!tar -xzf movies.tar.gz
!mv movies/val.jsonl movies/dev.jsonl

--2020-04-10 16:07:35--  http://www.eraserbenchmark.com/zipped/movies.tar.gz
Resolving www.eraserbenchmark.com (www.eraserbenchmark.com)... 52.5.68.140, 54.158.230.58, 35.168.162.120, ...
Connecting to www.eraserbenchmark.com (www.eraserbenchmark.com)|52.5.68.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3899487 (3.7M) [application/gzip]
Saving to: ‘movies.tar.gz.1’

movies.tar.gz.1     100%[===================>]   3.72M  --.-KB/s    in 0.1s    

2020-04-10 16:07:36 (37.8 MB/s) - ‘movies.tar.gz.1’ saved [3899487/3899487]



In [2]:
import json
from rationale_benchmark.utils import load_flattened_documents
documents = load_flattened_documents('movies', docids=None)
def read_annotations(json_file) :
    anns = [json.loads(line) for line in open(json_file)]
    for a in anns :
        doc_id = a['annotation_id']
        a['document'] = " ".join(documents[doc_id])
        a['label'] = a['classification']
        del a['classification']
        a['rationale'] = []
        for evgroup in a['evidences'] :
            for ev in evgroup :
                assert ev['docid'] == doc_id
                a['rationale'].append((ev['start_token'], ev['end_token']))
        del a['evidences']
        del a['query_type']
        del a['query']
        
    return anns

In [3]:
import os
os.makedirs('data/', exist_ok=True)

In [5]:
for key in ['train', 'dev', 'test'] :
    ann = read_annotations('movies/' + key + '.jsonl')
    with open('data/' + key + '.jsonl', 'w') as f :
        f.write('\n'.join([json.dumps(line) for line in ann]))